In [5]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score

In [6]:
def calculate_ece(confidences, accuracies, n_bins=10) -> float:
    """
    Calculate the expected calibration error (ECE) given a list of confidence scores (0-1) and accuracy scores (0 or 1).
    """
    df = pd.DataFrame({"conf": confidences, "acc": accuracies}).dropna()

    confidences = torch.tensor(df["conf"].tolist())
    accuracies = torch.tensor(df["acc"].tolist())
    bin_boundaries = torch.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = torch.zeros(1)
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # Calculated |confidence - accuracy| in each bin
        in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
        prop_in_bin = in_bin.float().mean()
        if prop_in_bin.item() > 0:
            accuracy_in_bin = accuracies[in_bin].float().mean()
            avg_confidence_in_bin = confidences[in_bin].mean()
            ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    
    return ece.item()

In [9]:
nan_indices = set()

for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file)

        # Find indices where either column has NaN
        indices_with_nan = df[
            df["extracted_answer"].isna() | df["verbal_numerical_confidence"].isna()
        ].index

        # Combine indices across all files
        nan_indices.update(indices_with_nan)

# Convert to sorted list if needed
nan_indices = sorted(nan_indices)
len(nan_indices)

4715

In [ ]:
stats_df = pd.DataFrame()
for file in os.listdir("../results"):
    if file.endswith("csv"):
        df = pd.read_csv("../results/" + file).drop(index=nan_indices).reset_index(drop=True)
        # df_no_na = df[["extracted_answer", "correct_answer", "verbal_numerical_confidence", "logit_perplexity_confidence", "verbal_linguistic_confidence"]].dropna()
        # acc_no_na = df[["extracted_answer", "correct_answer"]].dropna()
        acc_no_na = df
        new_row = pd.DataFrame({
            "Model": [file.replace("mmlu_pro_", "").replace("_eval_all_None.csv", "").strip().capitalize()],
            "Missing answer": [df["extracted_answer"].isna().sum()],
            "Missing verbal numerical confidence": [df["verbal_numerical_confidence"].isna().sum()],
            "Missing logit perplexity confidence": [df["logit_perplexity_confidence"].isna().sum()],
            "Missing verbal linguistic confidence": [df["verbal_linguistic_confidence"].isna().sum()],
            "Accuracy": [(df["extracted_answer"] == df["correct_answer"]).mean()],
            "Accuracy without na": [(acc_no_na["extracted_answer"] == acc_no_na["correct_answer"]).mean()],
            "ECE verbal numerical confidence": [calculate_ece(df["verbal_numerical_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE logit perplexity confidence": [calculate_ece(df["logit_perplexity_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "ECE verbal linguistic confidence": [calculate_ece(df["verbal_linguistic_confidence"].values, (df["extracted_answer"] == df["correct_answer"]))],
            "AUROC verbal numerical confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]).values, df["verbal_numerical_confidence"].fillna(0).values),
            "AUROC logit perplexity confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["logit_perplexity_confidence"].fillna(0).values),
            "AUROC verbal linguistic confidence": roc_auc_score((df["extracted_answer"] == df["correct_answer"]), df["verbal_linguistic_confidence"].fillna(0).values),
            "Mean verbal numerical confidence": [df["verbal_numerical_confidence"].mean()],
            "Mean logit perplexity confidence": [df["logit_perplexity_confidence"].mean()],
            "Mean verbal linguistic confidence": [df["verbal_linguistic_confidence"].mean()],
            "Std verbal numerical confidence": [df["verbal_numerical_confidence"].std()],
            "Std logit perplexity confidence": [df["logit_perplexity_confidence"].std()],
            "Std verbal linguistic confidence": [df["verbal_linguistic_confidence"].std()],
        })
        stats_df = pd.concat([stats_df, new_row], ignore_index=True)
    
display(stats_df.sort_values(by="Model", ignore_index=True))

,Model,Missing answer,Missing verbal numerical confidence,Missing logit perplexity confidence,Missing verbal linguistic confidence,Accuracy,Accuracy without na,ECE verbal numerical confidence,ECE logit perplexity confidence,ECE verbal linguistic confidence,AUROC verbal numerical confidence,AUROC logit perplexity confidence,AUROC verbal linguistic confidence,Mean verbal numerical confidence,Mean logit perplexity confidence,Mean verbal linguistic confidence,Std verbal numerical confidence,Std logit perplexity confidence,Std verbal linguistic confidence
0,Claude-3-5-haiku-20241022,0,0,7317,0,0.736641,0.736641,0.174961,0.000000,0.0,0.677942,0.500000,0.5,0.911356,NaN,0.0,0.061631,NaN,0.0
1,Claude-3-7-sonnet-20250219,0,0,7317,0,0.875222,0.875222,0.046156,0.000000,0.0,0.739587,0.500000,0.5,0.919943,NaN,0.0,0.081832,NaN,0.0
2,Claude-3-haiku-20240307,0,0,7317,0,0.529315,0.529315,0.389709,0.000000,0.0,0.521414,0.500000,0.5,0.919024,NaN,0.0,0.047096,NaN,0.0
3,Deepseek-chat,0,0,0,0,0.862102,0.862102,0.055658,0.137897,0.0,0.756998,0.498989,0.5,0.917405,0.999998,0.0,0.061322,0.000070,0.0
4,Deepseek-reasoner,0,0,7317,0,0.887386,0.887386,0.033654,0.000000,0.0,0.778729,0.500000,0.5,0.920849,NaN,0.0,0.078725,NaN,0.0
5,Gemini-1.5-flash,0,0,0,0,0.756321,0.756321,0.198570,0.172543,0.0,0.682826,0.698635,0.5,0.953443,0.928781,0.0,0.090277,0.046961,0.0
6,Gemini-1.5-flash-8b,0,0,0,0,0.656827,0.656827,0.271525,0.219067,0.0,0.599284,0.656910,0.5,0.926165,0.875893,0.0,0.105933,0.071131,0.0
7,Gemini-1.5-pro,0,0,0,0,0.837638,0.837638,0.125566,0.093005,0.0,0.676164,0.676572,0.5,0.962481,0.930643,0.0,0.066883,0.044069,0.0
8,Gemini-2.0-flash,0,0,0,0,0.852399,0.852399,0.122874,0.028569,0.0,0.719353,0.760090,0.5,0.973936,0.869650,0.0,0.065726,0.075660,0.0
9,Gemini-2.0-flash-lite,0,0,0,0,0.805248,0.805248,0.151665,0.060152,0.0,0.726906,0.757006,0.5,0.955929,0.863708,0.0,0.092141,0.078199,0.0
